# Fundamentals of Machine Learning: Data Analysis Task


# Y3857862











#Question 1:

In [1]:
import pandas as pd
import sys
from sklearn import linear_model, preprocessing
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,OneHotEncoder



data_unpacked = pd.read_csv("data.csv")

variables = ["X1", "X2", "X3", "X4", "X5"]
x = data_unpacked[variables]
y = data_unpacked["Y"]


alphas = [ 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2) 

std_scale = preprocessing.StandardScaler().fit(train_x)
train_x_std = std_scale.transform(train_x)
test_x_std = std_scale.transform(test_x)


lasso = linear_model.Lasso()
rdg = linear_model.Ridge()
lm = linear_model.LinearRegression()
rdgCV = RidgeCV(alphas=alphas)
lassoCV = LassoCV(alphas=alphas)

In [2]:
lassoCV.fit(train_x_std, train_y)
data_y_predict = lassoCV.predict(test_x_std)
print("Lasso CV (Standardised) \n")
print("Training set = 0.8, Test Set = 0.2 \n")
print("Predicted Y values: {0}\n".format(data_y_predict))
print("R^2: {0}\n".format(r2_score(test_y, data_y_predict)))
print("Lasso CV's Coefficients: {}\n".format(lassoCV.coef_))
print("Best Alpha: {}\n".format(lassoCV.alpha_))

Lasso CV (Standardised) 

Training set = 0.8, Test Set = 0.2 

Predicted Y values: [ 0.00528217  0.10402995  1.4522392   0.90214478  2.2907622   1.34305412
  1.85012317  0.37879465  2.00167838  1.23427442  0.76827834  0.63820125
  1.39490412  0.97329696  0.16314203  1.93230319  1.10657231  0.98844238
  1.19771978 -0.37130487]

R^2: 0.9807789579160302

Lasso CV's Coefficients: [-0.         -0.04872817  0.76492131  0.37978939  0.        ]

Best Alpha: 0.01



My Chosen Model is Lasso CV acting on standardised data.



I have evaluated an initial linear regression of 0.9977 for non-standardised inputs and 0.9982 for standardised inputs which represents a very strong linear relationship between the two variables.

Even though the linear model may be optimal for the data given to create the model, it is not necessarily guaranteed to be the best model for predictions on unseen data.

If our underlying data follows a relatively simple model, and the model implemented is too complex for the task, it will lead to applying too much weight on any possible change or variance in the data. A model that overreacts and overcompensates for even the slightest change in our data will be incorrect and problematic for any predictions. The liner model's high R-squared values exemplify the phenomenon that is overfitting. When you have features in your dataset that are highly linearly correlated with other features, turns out linear models will be likely to overfit.

Furthermore, the dataset is small and restricted, so it is more likely that you can achieve a high linear correlation between the two variables, thus the need to implement a form of regularisation to prevent overfitting.

By comparing the standardised linear regression to the non-standardised linear regression, I evaluate that standardisation of data may prove useful for the final model as it improved the accuracy of the prediction. Standardisation is particularly useful as it is more robust to outlier samples compared to normalisation and within the dataset, despite the high linear correlation, there were a few rows of data that were not linearly correlated.

There are two forms of regularisation with the linear model, and they are ridge regression and lasso regression.

Lasso regression - the algorithm aims to discount the extra features that doesn't have any or little use to the rest of the data. It is useful as we can train with less data, but the processing is a little bit harder.

Ridge regression, the algorithm aims to undermine the effect of the extra features that have little to no use to the rest of the data by reducing its effect on the final output, but ridge regression does not completely remove them from the dataset which is easier to process.

Lasso tends to perform better in a scenario where a relatively small number of predictors have substantial coefficients, and the remaining predictors have very small coefficients or that equal zero.

Ridge regression will perform better when the response is a function of many predictors, all with coefficients of roughly equal size. However, the number of predictors that is related to the response is never deduced for real data sets.

Cross-validation can be used in order to determine which approach is better on a particular data set by determining the best alpha coefficients appropriate for the specified dataset.

After evaluating my results above, I believe the best course of action is to implement a cross-validation lasso regression operating on standardised inputs.

The reasons why I chose this model is due to the fact that:

Lasso regression without cross-validation produced an R-Squared Score of 0.641 - this regression used an alpha coefficient of 1. However, there is a stark improvement with Lasso CV with an R-Squared score of 0.981 - an 0.24 improvement. The improvement in the score is due to lasso regression getting progressively worse as the alpha coefficient increases - evident with a sharp decline in the cross-validation scores.

Furthermore, I chose lasso regression to operate on standardised outputs due to the presence of a few anomalous data in the training data set within data.csv. I understand that Cross validated lasso regression produced a higher R^2 Score on non-standardised data with an R-Squared score of 0.9901 than the same regression on standardised data with an R-Squared value of 0.9802. Despite the higher R-Squared scores, there are cases of data that do not fit the higher linear correlation as stated so to make the model as robust as possible I am opting for operating on standardised data.

The reasons why I chose lasso regression over ridge regression is due to numerous reasons. Firstly, lasso penalises outliers more, and with an unfiltered dataset, there is a fairly high likelihood of anomalous data so to ensure the model is robust enough I aim to discount the possible effect of the outliers on the predictions.

Secondly, lasso regression performs better than ridge regression when there are a relatively small number of predictors with substantial coefficients, and the remaining predictors have very small coefficients or that equal zero. This is because lasso is Laplace distributed rather than normally distributed, like Ridge, making it easier for the coefficients to be zero and therefore easier to eliminate some of the outliers not contributing to the output. The coefficients for cross-validated lasso regression fit this profile, as shown above.

Lastly, ridge regression with cross-validation has infinitely small values close to zero and coefficient of zero, or close to zero, shows no meaningful relationship and fragile features. The minuscule values that are close to zero are not ideal for ridge regression as the function requires many predictors, all with coefficients of roughly equal size. The issue is that the coefficients are fairly varied and the dataset is too small to get many predictors.

In [3]:
data_unpacked = pd.read_csv("unseendata.csv")

variables = ["X1", "X2", "X3", "X4", "X5"]
x = data_unpacked[variables]
y = data_unpacked["Y"]


alphas = [ 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2) 

std_scale = preprocessing.StandardScaler().fit(train_x)
train_x_std = std_scale.transform(train_x)
test_x_std = std_scale.transform(test_x)

lassoCV = LassoCV(alphas=alphas)

lassoCV.fit(train_x_std, train_y)
data_y_predict = lassoCV.predict(test_x_std)
print("Lasso CV (Standardised) \n")
print("Training set = 0.8, Test Set = 0.2 \n")
print("Predicted Y values: {0}\n".format(data_y_predict))

Lasso CV (Standardised) 

Training set = 0.8, Test Set = 0.2 

Predicted Y values: [ 0.94340385  1.57182118  2.28020589 -0.34400294  1.578969    0.30191914
  1.076311   -0.10297852  0.17976204  0.07106091  0.61711449  1.20732254
  1.35546686  0.57737953  0.08192891  1.45778196  1.61108865  2.32247569
  1.50809692  0.61569069]

